In [1]:
cd ..

/home/vule/projects/learn_agent


In [2]:
# !pip install python-dotenv

In [4]:
import anthropic
# import src key
from src.key import claude_api_key

from dotenv import load_dotenv

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=claude_api_key
)

model="claude-3-sonnet-20240229"
small_model="claude-3-haiku-20240307"

### A failed attempt

In [6]:
# A relatively simple math problem
response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content":"Multiply 1984135 by 9343116. Only respond with the result"}],
    max_tokens=400
)
print(response.content[0].text)

18,564,825,366,360


### Step 1: Tools and prompt

In [9]:
# create a function calculater with parameters opt, a and b
# opt is a string, either "add", "sub", "mul", "div"
# a and b are numbers
# return the result of the operation

def calculator(opt, a, b):
    if opt == "add":
        return a + b
    elif opt == "sub":
        return a - b
    elif opt == "mul":
        return a * b
    elif opt == "div":
        return a / b
    else:
        return "Invalid operation"
    
# give a test cases
# print(calculator("add", 1, 2))
# print(calculator("sub", 1, 2))
# print(calculator("mul", 1, 2))
# print(calculator("div", 1, 2))
# print(calculator("invalid", 1, 2))

# measure tool usage
# name, description, input_schema

calculator_tool = {
    "name": "calculator",
    "description": "A calculator that can perform basic arithmetic operations",
    "input_schema": {
        "type": "object",
        "properties": {
            "opt": {
                "type": "string",
                "enum": ["add", "sub", "mul", "div"],
                "description": "The operation to perform"
            },
            "a": {
                "type": "number",
                "description": "The first number to operate on"
            },
            "b": {
                "type": "number",
                "description": "The second number to operate on"
            }
        },
        "required": ["opt", "a", "b"],
    },
}

In [11]:
# step 2: Claude uses the tool
response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content": "Multiply 1984135 by 9343116. Only respond with the result"}],
    max_tokens=300,
    # Tell Claude about our tool
    tools=[calculator_tool]
)
response

Message(id='msg_01XqDNxEyjFfe3Bkjr5CrLKF', content=[ToolUseBlock(id='toolu_019ryxciBRx53fgyy41DguNu', input={'a': 1984135, 'b': 9343116, 'opt': 'mul'}, name='calculator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=415, output_tokens=89))

In [16]:
# step 3: Claude uses the output to answer the user
tool_name = response.content[0].name
tool_inputs = response.content[0].input
tool_outputs = calculator(**tool_inputs)

print("The Tool Name Claude Wants To Call:", tool_name)
print("The Inputs Claude Wants To Call It With:", tool_inputs)

The Tool Name Claude Wants To Call: calculator
The Inputs Claude Wants To Call It With: {'a': 1984135, 'b': 9343116, 'opt': 'mul'}


In [17]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content":"What color are emeralds?"}],
    max_tokens=400,
    tools=[calculator_tool]
)
response    

Message(id='msg_014tHq6brcMmihZYYXuxyx85', content=[ToolUseBlock(id='toolu_019fimhboAYQj8Q3d9rhPvPm', input={'a': 0, 'b': 0, 'opt': 'add'}, name='calculator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=404, output_tokens=85))

In [19]:
# add system prompt
response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content":"What color are emeralds?"}],
    max_tokens=400,
    tools=[calculator_tool],
    system="You are a helpful assistant that can perform basic arithmetic operations. You have access to tools, but only use them when necessary. If a tool is not required, respond as normal"
)
print(response)
print(response.stop_reason)



Message(id='msg_01PvK3vbAS4fmR8NJ8QtJDGK', content=[TextBlock(text='Emeralds are green in color. Emeralds are a variety of the mineral beryl that get their green color from traces of chromium and sometimes vanadium.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=440, output_tokens=40))
end_turn


### Putting it all together

In [35]:
# Step 1: Tools and prompt
def calculator(opt, a, b):
    if opt == "add":
        return a + b
    elif opt == "sub":
        return a - b
    elif opt == "mul":
        return a * b
    elif opt == "div":
        return a / b
    else:
        return "Invalid operation"

calculator_tool = {
    "name": "calculator",
    "description": "A calculator that can perform basic arithmetic operations",
    "input_schema": {
        "type": "object",
        "properties": {
            "opt": {
                "type": "string",
                "enum": ["add", "sub", "mul", "div"],
                "description": "The operation to perform"
            },
            "a": {
                "type": "number",
                "description": "The first number to operate on"
            },
            "b": {
                "type": "number",
                "description": "The second number to operate on"
            }
        },
        "required": ["opt", "a", "b"],
    },
}
def prompt_claude(user_message, tools):
    system_prompt = """
    You have access to tools, but only use them when necessary. If a tool is not required, respond as normal
    """
    # Step 2: Claude uses the tool
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        messages=[{"role": "user", "content": user_message}],
        max_tokens=400,
        tools=tools,
        system=system_prompt
    )
    print(response)
    print(response.stop_reason)
    # Step 3: Claude uses the output to answer the user
    if response.stop_reason == "tool_use":
        tool_name = response.content[-1].name
        tool_inputs = response.content[-1].input
        try:
            if tool_name == "calculator":
                tool_outputs = calculator(**tool_inputs)
                print("The Tool Name Claude Wants To Call:", tool_name)
                print("The Inputs Claude Wants To Call It With:", tool_inputs)
                print("The Output Of The Tool:", tool_outputs)
        except Exception as e:
            print("An error occurred while calling the tool:", e)
            print("The Tool Name Claude Wants To Call:", tool_name)
            print("The Inputs Claude Wants To Call It With:", tool_inputs)
            print("The Output Of The Tool:", tool_outputs)
    elif response.stop_reason == "end_turn":
        print("No tool was used")
        print("The Response:", response)


prompt_claude("What is 3 times 2?", [calculator_tool])

Message(id='msg_01BcPW8ti7AnG7QNzCUkUGaE', content=[TextBlock(text="I'll help you calculate that using the calculator.", type='text'), ToolUseBlock(id='toolu_01SgdjTBbSrn6G3CeQboUQog', input={'opt': 'mul', 'a': 3, 'b': 2}, name='calculator', type='tool_use')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=430, output_tokens=96))
tool_use
The Tool Name Claude Wants To Call: calculator
The Inputs Claude Wants To Call It With: {'opt': 'mul', 'a': 3, 'b': 2}
The Output Of The Tool: 6


In [34]:
# print all model names
# for model in client.models.list():
#     print(model.id)

In [38]:
# strick model to return expected output format by using tools input
print_tool = {
    "name": "print_sentiment_scores",
    "description": "Prints the sentiment scores of a given text.",
    "input_schema": {
        "type": "object",
        "properties": {
            "positive_score": {"type": "number", "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
            "negative_score": {"type": "number", "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
            "neutral_score": {"type": "number", "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
        },
        "required": ["positive_score", "negative_score", "neutral_score"]
    }
}
tools = [print_tool]
prompt = "What is the sentiment score of the text 'I love programming'?"
response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=400,
        tools=tools
    )
print(response)
print(response.stop_reason)
print(response.content[-1].input)

Message(id='msg_01CVdhfAKwpxLn3TnabXLQes', content=[TextBlock(text="I'll help you determine the sentiment score for the text 'I love programming' using the print_sentiment_scores function.", type='text'), ToolUseBlock(id='toolu_017Wm9Git2yFc5q58W3sWCwj', input={'positive_score': 0.875, 'negative_score': 0.125, 'neutral_score': 0.0}, name='print_sentiment_scores', type='tool_use')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=449, output_tokens=128))
tool_use
ToolUseBlock(id='toolu_017Wm9Git2yFc5q58W3sWCwj', input={'positive_score': 0.875, 'negative_score': 0.125, 'neutral_score': 0.0}, name='print_sentiment_scores', type='tool_use')


In [40]:
tweet = "I'm a HUGE hater of pickles.  I actually despise pickles.  They are garbage."
system_prompt = "Allways user print_tool"
query = f"""
<text>
{tweet}
</text>
"""
response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        messages=[{"role": "user", "content": query}],
        max_tokens=400,
        tools=tools,
        system=system_prompt
    )
print(response)
print(response.stop_reason)
print(response.content[-1].input)

Message(id='msg_01TppJyws1Bu73WJt4MZMrD9', content=[TextBlock(text="I'll analyze the sentiment of this text using the print_sentiment_scores tool:", type='text'), ToolUseBlock(id='toolu_01Uv9pHMdxrQ1MHHHnZsioVN', input={'positive_score': 0.0, 'negative_score': 0.9, 'neutral_score': 0.1}, name='print_sentiment_scores', type='tool_use')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=476, output_tokens=119))
tool_use
{'positive_score': 0.0, 'negative_score': 0.9, 'neutral_score': 0.1}


In [44]:
# force the model to use the tool
tweet = "What the fuck is this?"
query = f"""
<text>
{tweet}
</text>
"""

response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        messages=[{"role": "user", "content": query}],
        max_tokens=400,
        tools=tools,
        tool_choice={"type": "tool", "name": "print_sentiment_scores"}
    )
print(response)
print(response.stop_reason)
print(response.content[-1].input)

Message(id='msg_019wsoyUaNZoG3CuFgbcnYmT', content=[ToolUseBlock(id='toolu_01JvBUSbpwyrkMMQzNowMtnF', input={'positive_score': 0.0, 'negative_score': 1.0, 'neutral_score': 0.0}, name='print_sentiment_scores', type='tool_use')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=548, output_tokens=79))
tool_use
{'positive_score': 0.0, 'negative_score': 1.0, 'neutral_score': 0.0}


In [45]:
response.content

[ToolUseBlock(id='toolu_01JvBUSbpwyrkMMQzNowMtnF', input={'positive_score': 0.0, 'negative_score': 1.0, 'neutral_score': 0.0}, name='print_sentiment_scores', type='tool_use')]

In [ ]:
https://github.com/anthropics/courses/blob/master/tool_use/03_structured_outputs.ipynb